<h1>Whole Implementation of DeBlurGan using Tensorflow and Keras</h1>

organize_gopro_dataset.py

In [13]:
import os
from shutil import copyfile

import click
import tqdm

def reorganize_gopro_files(dir_in, dir_out):
    if not os.path.exists(dir_out):
        os.makedirs(dir_out)

    for folder_train_test in tqdm.tqdm(os.listdir(dir_in), desc='dir'):
        output_directory = os.path.join(dir_out, folder_train_test)
        output_directory_A = os.path.join(output_directory, 'A')
        output_directory_B = os.path.join(output_directory, 'B')
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        if not os.path.exists(output_directory_A):
            os.makedirs(output_directory_A)
        if not os.path.exists(output_directory_B):
            os.makedirs(output_directory_B)

        current_folder_path = os.path.join(dir_in, folder_train_test)
        for image_folder in tqdm.tqdm(os.listdir(current_folder_path), desc='image_folders'):

            current_sub_folder_path = os.path.join(
                current_folder_path, image_folder)

            for image_blurred in os.listdir(os.path.join(current_sub_folder_path, 'blur')):
                current_image_blurred_path = os.path.join(
                    current_sub_folder_path, 'blur', image_blurred)
                output_image_blurred_path = os.path.join(
                    output_directory_A, image_folder + "_" + image_blurred)
                copyfile(current_image_blurred_path, output_image_blurred_path)

            for image_sharp in os.listdir(os.path.join(current_sub_folder_path, 'sharp')):
                current_image_sharp_path = os.path.join(
                    current_sub_folder_path, 'sharp', image_sharp)
                output_image_sharp_path = os.path.join(
                    output_directory_B, image_folder + "_" + image_sharp)
                copyfile(current_image_sharp_path, output_image_sharp_path)


if __name__ == "__main__":
    reorganize_gopro_files('/kaggle/input/gopro-data', '/kaggle/working/')


dir: 100%|██████████| 2/2 [01:52<00:00, 56.22s/it]


layer_utils.py

In [1]:
import tensorflow as tf

from keras.models import Model
from keras.layers import InputSpec, Layer
from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.layers import Add
from keras.utils import conv_utils
from keras.layers.core import Dropout


def res_block(input, filters, kernel_size=(3, 3), strides=(1, 1), use_dropout=False):
    """
    Instanciate a Keras Resnet Block using sequential API.

    :param input: Input tensor
    :param filters: Number of filters to use
    :param kernel_size: Shape of the kernel for the convolution
    :param strides: Shape of the strides for the convolution
    :param use_dropout: Boolean value to determine the use of dropout
    :return: Keras Model
    """
    x = ReflectionPadding2D((1, 1))(input)
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
        x = Dropout(0.5)(x)

    x = ReflectionPadding2D((1, 1))(x)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)

    merged = Add()([input, x])
    return merged


def spatial_reflection_2d_padding(x, padding=((1, 1), (1, 1)), data_format=None):
    """
    Pad the 2nd and 3rd dimensions of a 4D tensor.

    :param x: Input tensor
    :param padding: Shape of padding to use
    :param data_format: Tensorflow vs Theano convention ('channels_last', 'channels_first')
    :return: Tensorflow tensor
    """
    assert len(padding) == 2
    assert len(padding[0]) == 2
    assert len(padding[1]) == 2
    if data_format is None:
        data_format = image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format ' + str(data_format))

    if data_format == 'channels_first':
        pattern = [[0, 0],
                   [0, 0],
                   list(padding[0]),
                   list(padding[1])]
    else:
        pattern = [[0, 0],
                   list(padding[0]), list(padding[1]),
                   [0, 0]]
    return tf.pad(x, pattern, "REFLECT")


class ReflectionPadding2D(Layer):
    """Reflection-padding layer for 2D input (e.g. picture).
    This layer can add rows and columns or zeros
    at the top, bottom, left and right side of an image tensor.
    # Arguments
        padding: int, or tuple of 2 ints, or tuple of 2 tuples of 2 ints.
            - If int: the same symmetric padding is applied to width and height.
            - If tuple of 2 ints:
                interpreted as two different
                symmetric padding values for height and width:
                `(symmetric_height_pad, symmetric_width_pad)`.
            - If tuple of 2 tuples of 2 ints:
                interpreted as
                `((top_pad, bottom_pad), (left_pad, right_pad))`
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
    # Input shape
        4D tensor with shape:
        - If `data_format` is `"channels_last"`:
            `(batch, rows, cols, channels)`
        - If `data_format` is `"channels_first"`:
            `(batch, channels, rows, cols)`
    # Output shape
        4D tensor with shape:
        - If `data_format` is `"channels_last"`:
            `(batch, padded_rows, padded_cols, channels)`
        - If `data_format` is `"channels_first"`:
            `(batch, channels, padded_rows, padded_cols)`
    """

    def __init__(self, padding=(1, 1), data_format=None, **kwargs):
        super(ReflectionPadding2D, self).__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)

        if isinstance(padding, int):
            self.padding = ((padding, padding), (padding, padding))

        elif hasattr(padding, '__len__'):

            if len(padding) != 2:
                raise ValueError(
                    '`padding` should have two elements. ' 'Found: ' + str(padding))
            height_padding = conv_utils.normalize_tuple(padding[0], 2,
                                                        '1st entry of padding')
            width_padding = conv_utils.normalize_tuple(padding[1], 2,
                                                       '2nd entry of padding')
            self.padding = (height_padding, width_padding)
        else:
            raise ValueError('`padding` should be either an int, '
                             'a tuple of 2 ints '
                             '(symmetric_height_pad, symmetric_width_pad), '
                             'or a tuple of 2 tuples of 2 ints '
                             '((top_pad, bottom_pad), (left_pad, right_pad)). '
                             'Found: ' + str(padding))
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            if input_shape[2] is not None:
                rows = input_shape[2] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None

            if input_shape[3] is not None:
                cols = input_shape[3] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None

            return (input_shape[0], input_shape[1], rows, cols)

        elif self.data_format == 'channels_last':
            if input_shape[1] is not None:
                rows = input_shape[1] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None

            if input_shape[2] is not None:
                cols = input_shape[2] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None

            return (input_shape[0], rows, cols, input_shape[3])

    def call(self, inputs):
        return spatial_reflection_2d_padding(inputs, padding=self.padding, data_format=self.data_format)

    def get_config(self):
        config = {'padding': self.padding, 'data_format': self.data_format}
        base_config = super(ReflectionPadding2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


if __name__ == "__main__":
    input = Input(shape=(256, 256, 3))
    x = ReflectionPadding2D(3)(input)
    model = Model(input, x)
    model.summary()


2023-07-15 23:37:57.873118: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-15 23:37:57.916071: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-15 23:37:57.916087: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 reflection_padding2d (Refle  (None, 262, 262, 3)      0         
 ctionPadding2D)                                                 
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


2023-07-15 23:37:59.946300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-15 23:37:59.946317: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-15 23:37:59.946328: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-07-15 23:37:59.946955: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


losses.py

In [2]:
import keras.backend as k
from keras.applications.vgg16 import VGG16
from keras.models import Model
import numpy as np

# Note the image_shape must be multiple of patch_shape
image_shape = (256, 256, 3)


def l1_loss(y_true, y_pred):
    return k.mean(k.abs(y_pred - y_true))

vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
# vgg_variables = vgg.trainable_variables
# loss_model = Model(vgg_variables)

loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
loss_model.trainable = False

@tf.function
def perceptual_loss(y_true, y_pred):
    features1 = loss_model(y_true)
    features2 = loss_model(y_pred)
    loss = tf.reduce_mean(tf.square(features1 - features2))
    return loss

def perceptual_loss_100(y_true, y_pred):
    return 100 * perceptual_loss(y_true, y_pred)


def wasserstein_loss(y_true, y_pred):
    return k.mean(y_true*y_pred)


def gradient_penalty_loss(self, y_ture, y_pred, averaged_samples):
    gradients = k.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = k.square(gradients)
    gradients_sqr_sum = k.sum(
        gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = k.sqrt(gradients_sqr_sum)
    gradient_penalty = k.square(1 - gradient_l2_norm)
    return k.mean(gradient_penalty)

58889256/58889256 [==============================] - 174s 3us/step


model.py

In [3]:
from tensorflow import keras
from keras.layers import Input, Activation, Add, UpSampling2D, LeakyReLU, Conv2D
from keras.layers.core import Dense, Flatten, Lambda
from keras.layers import BatchNormalization
from keras.models import Model

# The paper defined hyper-parameter: chr
channel_rate = 64
# Note the image_sape must be multiplied of patch_shape
image_shape = (256, 256, 3)
patch_shape = (channel_rate, channel_rate, 3)

ngf = 64
ndf = 64
input_nc = 3
output_nc = 3
input_shape_generator = (256, 256, input_nc)
input_shape_discriminator = (256, 256, output_nc)
n_block_gen = 9


def generator_model():
    """Build generator architecture.."""

    inputs = Input(shape=image_shape)

    x = ReflectionPadding2D((3, 3))(inputs)
    x = Conv2D(filters=ngf, kernel_size=(7, 7), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    n_downsampling = 2
    for i in range(n_downsampling):
        mult = 2**i
        x = Conv2D(filters=ngf*mult*2, kernel_size=(3, 3),
                   strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    mult = 2**n_downsampling
    for i in range(n_block_gen):
        x = res_block(x, ngf*mult, use_dropout=True)

    for i in range(n_downsampling):
        mult = 2**(n_downsampling - i)
        # x = Conv2DTranspose(filters=int(nfg*mult / 2), kernel_size=(3, 3), strides=2, padding='same')(x)
        x = UpSampling2D()(x)
        x = Conv2D(filters=int(ngf*mult / 2),
                   kernel_size=(3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = ReflectionPadding2D((3, 3))(x)
    x = Conv2D(filters=output_nc, kernel_size=(7, 7), padding='valid')(x)
    x = Activation('tanh')(x)

    outputs = Add()([x, inputs])
    outputs = keras.layers.Lambda(lambda z: z/2)(outputs)

    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model


def discriminator_model():
    """Build discriminator architecture."""

    n_layers, use_sigmoid = 3, False
    inputs = Input(shape=input_shape_discriminator)

    x = Conv2D(filters=ndf, kernel_size=(4, 4),
               strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2**n, 8)
        x = Conv2D(filters=ndf*nf_mult, kernel_size=(4, 4),
                   strides=1, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2**n_layers, 8)
    x = Conv2D(filters=ndf*nf_mult, kernel_size=(4, 4),
               strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=1, kernel_size=(4, 4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model


def generator_containing_discriminator(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_image = generator(inputs)
    outputs = discriminator(generated_image)
    model = Model(inputs=inputs, outputs=outputs)
    return model


def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_image = generator(inputs)
    outputs = discriminator(generated_image)
    model = Model(inputs=inputs, outputs=[generated_image, outputs])
    return model


if __name__ == '__main__':
    g = generator_model()
    g.summary()
    d = discriminator_model()
    d.summary()
    m = generator_containing_discriminator(
        generator_model(), discriminator_model())
    m.summary()


Model: "Generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 reflection_padding2d_1 (Reflec  (None, 262, 262, 3)  0          ['input_3[0][0]']                
 tionPadding2D)                                                                                   
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  9472        ['reflection_padding2d_1[0][0]'] 
                                )                                                         

utils.py

In [4]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf


RESHAPE = (256, 256)


def is_an_image_file(filename):
    IMAGE_EXTENSIONS = ['.png', '.jpg', '.jpeg']
    for ext in IMAGE_EXTENSIONS:
        if ext in filename:
            return True
    return False


def list_image_files(directory):
    files = sorted(os.listdir(directory))
    return [os.path.join(directory, f) for f in files if is_an_image_file(f)]


def load_image(path):
    img = Image.open(path)
    return img


def preprocess_image(cv_img):
    cv_img = cv_img.resize(RESHAPE)
    img = np.array(cv_img)
    img = (img - 127.5) / 127.5
    return img


def deprocess_image(img):
    img = img * 127.5 + 127.5
    return img.astype('uint8')


def save_image(np_arr, path):
    img = np_arr * 127.5 + 127.5
    im = Image.fromarray(img)
    im.save(path)


def load_images(path, n_images):
    if n_images < 0:
        n_images = float("inf")
    A_paths, B_paths = os.path.join(path, 'A'), os.path.join(path, 'B')
    all_A_paths, all_B_paths = list_image_files(
        A_paths), list_image_files(B_paths)
    images_A, images_B = [], []
    images_A_paths, images_B_paths = [], []
    for path_A, path_B in zip(all_A_paths, all_B_paths):
        img_A, img_B = load_image(path_A), load_image(path_B)
        images_A.append(preprocess_image(img_A))
        images_B.append(preprocess_image(img_B))
        images_A_paths.append(path_A)
        images_B_paths.append(path_B)
        if len(images_A) > n_images - 1:
            break

    return {
        'A': np.array(images_A),
        'A_paths': np.array(images_A_paths),
        'B': np.array(images_B),
        'B_paths': np.array(images_B_paths)
    }


def write_log(callback, names, logs, batch_no):
    """
    Util to write callback for Keras training
    """
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()

In [22]:
import os
import datetime
import numpy as np
import tqdm
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

BASE_DIR = '/kaggle/working/training/'


def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(
        epoch_number, current_loss)), True)
    d.save_weights(os.path.join(
        save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)


def train_multiple_outputs(n_images, batch_size, log_dir, epoch_num, critic_updates=5):
    data = load_images('/kaggle/working/train', n_images)
    y_train, x_train = data['B'], data['A']

    g = generator_model()
    d = discriminator_model()
    d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

    d_opt = Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_on_g_opt = Adam(learning_rate=1E-4, beta_1=0.9,
                      beta_2=0.999, epsilon=1e-08)

    d.trainable = True
    d.compile(optimizer=d_opt, loss=wasserstein_loss)
    d.trainable = False
    loss = [perceptual_loss, wasserstein_loss]
    loss_weights = [100, 1]
    d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
    d.trainable = True
    
    output_true_batch, output_false_batch = np.ones(
        (batch_size, 1)), -np.ones((batch_size, 1))

    log_path = 'kaggle/working/logs'
    tensorboard_callbacks = TensorBoard(log_path)

    for epoch in tqdm.tqdm(range(epoch_num)):
        permutated_indexes = np.random.permutation(x_train.shape[0])

        d_losses = []
        d_on_g_losses = []
        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index *
                                               batch_size:(index+1)*batch_size]
            image_blur_batch = x_train[batch_indexes]
            image_full_batch = y_train[batch_indexes]

            generated_images = g.predict(
                x=image_blur_batch, batch_size=batch_size)

            for _ in range(critic_updates):
                d_loss_real = d.train_on_batch(
                    image_blur_batch, output_true_batch)
                d_loss_fake = d.train_on_batch(
                    generated_images, output_false_batch)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                d_losses.append(d_loss)

            d.trainable = False

            d_on_g_loss = d_on_g.train_on_batch(
                image_blur_batch, [image_full_batch, output_true_batch])
            d_on_g_losses.append(d_on_g_loss)

            d.trainable = True

            print(np.mean(d_losses), np.mean(d_on_g_losses))
            with open('long.txt', 'a+') as f:
                f.write('{} - {}\n'.format(epoch,
                        np.mean(d_losses), np.mean(d_on_g_losses)))

            save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))


def train_command(n_images, batch_size, log_dir, epoch_num, critic_updates):
    return train_multiple_outputs(n_images, batch_size, log_dir, epoch_num, critic_updates)


if __name__ == '__main__':
    train_command(512, 16, False, 4, 5)

  0%|          | 0/4 [00:00<?, ?it/s]

1/1 [==============================] - 0s 490ms/step


2023-07-15 17:23:49.031111: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_13/Generator/dropout_90/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


0.05560425270814449 11285.963889698187
1/1 [==============================] - 0s 31ms/step
0.02544535422930494 10581.75398670137
1/1 [==============================] - 0s 32ms/step
0.010465529724024236 9768.03116800083
1/1 [==============================] - 0s 34ms/step
-0.003207755711628124 9381.528065535129
1/1 [==============================] - 0s 33ms/step
-0.023478729627095164 8877.06560428357
1/1 [==============================] - 0s 32ms/step
-0.03970440891183292 8597.165293066171
1/1 [==============================] - 0s 32ms/step
-0.05010527684685907 8332.808982947157
1/1 [==============================] - 0s 27ms/step
-0.06113904242811259 7993.4636221789815
1/1 [==============================] - 0s 43ms/step
-0.0738611639658403 7740.56824566456
1/1 [==============================] - 0s 27ms/step
-0.09510312746046111 7519.48490844363
1/1 [==============================] - 0s 27ms/step
-0.10643078585536304 7359.623172093613
1/1 [==============================] - 0s 27ms/step
-0

 25%|██▌       | 1/4 [03:39<10:59, 219.98s/it]

...

 50%|█████     | 2/4 [07:11<07:09, 214.94s/it]

1/1 [==============================] - 0s 27ms/step
-0.49999988079054614 3901.4308990240097
1/1 [==============================] - 0s 28ms/step
-0.499999909876833 3798.367053886255
1/1 [==============================] - 0s 28ms/step
-0.49999993991779246 3562.2982198993363
1/1 [==============================] - 0s 26ms/step
-0.49999991768544094 3616.8577636828027
1/1 [==============================] - 0s 28ms/step
-0.4894552236163684 3627.6902217904726
1/1 [==============================] - 0s 28ms/step
-0.4768678785305721 3607.493870112631
1/1 [==============================] - 0s 26ms/step
-0.4777159062980439 3529.6170856867516
1/1 [==============================] - 0s 28ms/step
-0.4804625267251274 3589.0509421105185
1/1 [==============================] - 0s 28ms/step
-0.4826333570881462 3560.0145523349443
1/1 [==============================] - 0s 27ms/step
-0.48376357034220385 3559.1253625154495
1/1 [==============================] - 0s 27ms/step
-0.4846201697171237 3499.501212172435

 75%|███████▌  | 3/4 [10:44<03:34, 214.23s/it]

1/1 [==============================] - 0s 27ms/step
-0.49869652395725267 3186.0610376993814
1/1 [==============================] - 0s 26ms/step
-0.4991906762385329 2811.7962443331876
1/1 [==============================] - 0s 27ms/step
-0.49945940178393927 2992.5558526979553
1/1 [==============================] - 0s 28ms/step
-0.4995945513156199 2934.248821551601
1/1 [==============================] - 0s 27ms/step
-0.4996756410327152 2887.9500387271246
1/1 [==============================] - 0s 27ms/step
-0.49943407076320223 2880.56840634346
1/1 [==============================] - 0s 27ms/step
-0.4995149177942201 2862.362172115417
1/1 [==============================] - 0s 26ms/step
-0.49957516869701235 2846.9195943921804
1/1 [==============================] - 0s 29ms/step
-0.4996223721748705 2855.187799482434
1/1 [==============================] - 0s 27ms/step
-0.49965316434238816 2857.293266113599
1/1 [==============================] - 0s 27ms/step
-0.49968468760311496 2895.875424823978


100%|██████████| 4/4 [14:16<00:00, 214.20s/it]


test.py

In [26]:
import numpy as np
from PIL import Image

def test(batch_size):
    data = load_images('/kaggle/working/test', batch_size)
    y_test, x_test = data['B'], data['A']
    g = generator_model()
    g.load_weights('/kaggle/working/kaggle/working/training/715/generator_3_2811.h5')
    generator_images = g.predict(x=x_test, batch_size=batch_size)
    generated = np.array([deprocess_image(img) for img in generator_images])

    x_test = deprocess_image(x_test)
    y_test = deprocess_image(y_test)

    for i in range(generator_images.shape[0]):
        y = y_test[i, :, :, :]
        x = x_test[i, :, :, :]
        img = generated[i, :, :, :]
        output = np.concatenate((y, x, img), axis=1)
        im = Image.fromarray(output.astype(np.uint8))
        im.save('results{}.png'.format(i))


def test_command(batch_size):
    return test(batch_size)


if __name__ == '__main__':
    test_command(4)

1/1 [==============================] - 1s 1s/step


deblur_image.py

In [20]:
import numpy as np
from PIL import Image
import os


def deblur(weight_path, input_dir, output_dir):
    g = generator_model()
    g.load_weights(weight_path)
    for image_name in os.listdir(input_dir):
        image = np.array(
            [preprocess_image(load_image(os.path.join(input_dir, image_name)))])
        x_test = image
        generated_images = g.predict(x=x_test)
        generated = np.array([deprocess_image(img)
                             for img in generated_images])
        x_test = deprocess_image(x_test)
        for i in range(generated_images.shape[0]):
            x = x_test[i, :, :, :]
            img = generated[i, :, :, :]
            output = np.concatenate((x, img), axis=1)
            im = Image.fromarray(output.astype(np.uint8))
            im.save(os.path.join(output_dir, image_name))


def deblur_command(weight_path, input_dir, output_dir):
    return deblur(weight_path, input_dir, output_dir)


if __name__ == "__main__":
    deblur_command(weight_path='./generator_3_2811.h5',
                   input_dir='../demo/input', output_dir='../demo/output/')


1/1 [==============================] - 1s 592ms/step
